# Telco Cusstomer Churn: Model Training

### **Do Feature importance for numerical or corr or vif**
### **Do W or Importnace for cat cols or chi sql test**
### **Target encoding for more than 2 and if 2 label encoding**
### **in model training use smotek to undersample or oversample** 
### **Use Optuna**
### ***Target >80%**

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [19]:
import pandas as pd

# Read the saved datasets
X_train = pd.read_csv("artifacts/feature_engineering/X_train.csv")
X_test  = pd.read_csv("artifacts/feature_engineering/X_test.csv")
y_train = pd.read_csv("artifacts/feature_engineering/y_train.csv")
y_test  = pd.read_csv("artifacts/feature_engineering/y_test.csv")

# y_test = y_test.values.ravel()
# y_train = y_train.values.ravel()

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (5587, 20), y_train: (5587, 1)
X_test: (1397, 20), y_test: (1397, 1)


In [20]:
X_train.head()

,City,Contract,is_payment_recurring,Tenure Months,Tech Support,Online Security,Internet Service,Payment Method,Online Backup,Device Protection,Dependents,Streaming TV,Streaming Movies,is_payment_automatic,Monthly Charges,Paperless Billing,additional_services,Senior Citizen,Partner,CLTV
0,0.302354,0.413065,1,0.458333,0.139233,0.399282,0.176836,0.438043,0.209677,0.212866,0,0.287872,0.323001,0,0.315920,1,3,1,1,0.799867
1,0.250849,0.028169,0,0.791667,0.403853,0.141593,0.176836,0.154400,0.381595,0.212866,0,0.287872,0.288183,1,0.558209,1,2,0,0,0.788303
2,0.254207,0.099914,0,0.666667,0.403853,0.399282,0.408549,0.438043,0.209677,0.378411,0,0.287872,0.288183,0,0.803483,1,1,1,0,0.235268
3,0.270541,0.099914,0,0.944444,0.403853,0.141593,0.408549,0.154038,0.209677,0.378411,0,0.322954,0.288183,1,0.779104,1,2,1,1,0.879253
4,0.267147,0.099914,0,0.819444,0.139233,0.141593,0.176836,0.185957,0.381595,0.378411,0,0.322954,0.288183,0,0.505970,1,2,0,1,0.614187


In [64]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier 

def objective(trial):
    classifier = trial.suggest_categorical("classifier", [
        "LogisticRegression", "RandomForest", "GradientBoosting", "KNN", "XGBoost"
    ])

    if classifier == "LogisticRegression":
        param = {
            'C': trial.suggest_float('C', 1e-4, 1e4, log=True), 
            'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga', 'newton-cg']), 
            'tol': trial.suggest_float('tol', 1e-6, 1e-1, log=True), 
            'class_weight': trial.suggest_categorical('lg_class_weight', [None, 'balanced'])
        }
        model = LogisticRegression(**param)

    elif classifier == "RandomForest":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'max_depth': trial.suggest_int("max_depth", 5, 30),
            'min_samples_split': trial.suggest_int("min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
            'random_state': 42,
            'n_jobs': -1
        }
        model = RandomForestClassifier(**param)

    elif classifier == "GradientBoosting":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int("max_depth", 2, 10),
            'min_samples_split': trial.suggest_int("min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
            'subsample': trial.suggest_float("subsample", 0.6, 1.0),
            'max_features': trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
            'random_state': 42
        }
        model = GradientBoostingClassifier(**param)

    # elif classifier == "SVC":
    #     param = {
    #         'C': trial.suggest_float("C", 0.1, 10, log=True),
    #         'kernel': trial.suggest_categorical("kernel", ["linear", "rbf"]),
    #         'gamma': trial.suggest_categorical("gamma", ["scale", "auto"]),
    #         'probability': True,
    #         'random_state': 42
    #     }
    #     model = SVC(**param)

    elif classifier == "KNN":
        param = {
            'n_neighbors': trial.suggest_int("n_neighbors", 3, 15),
            'weights': trial.suggest_categorical("weights", ["uniform", "distance"]),
            'metric': trial.suggest_categorical("metric", ["euclidean", "manhattan"])
        }
        model = KNeighborsClassifier(**param)

    elif classifier == "XGBoost":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'max_depth': trial.suggest_int("max_depth", 3, 10),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            'subsample': trial.suggest_float("subsample", 0.6, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
            'gamma': trial.suggest_float("gamma", 0, 5),
            'reg_alpha': trial.suggest_float("reg_alpha", 1e-4, 10, log=True),
            'reg_lambda': trial.suggest_float("reg_lambda", 1e-4, 10, log=True),
            'use_label_encoder': False,
            'eval_metric': 'logloss',
            'random_state': 42,
            'n_jobs': -1
        }
        model = XGBClassifier(**param)
        
    f1_scorer = make_scorer(f1_score, average='macro')
    return cross_val_score(model, X_train, y_train, cv=5, scoring=f1_scorer, n_jobs=-1).mean()

# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=60)

trial = study.best_trial

print("F1 Score: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))


[I 2025-09-13 00:13:34,858] A new study created in memory with name: no-name-7d3502c3-e0c6-49ea-8de1-64627974dce8
[I 2025-09-13 00:13:35,308] Trial 0 finished with value: 0.7969121449671592 and parameters: {'classifier': 'XGBoost', 'n_estimators': 115, 'max_depth': 4, 'learning_rate': 0.12628095688621827, 'subsample': 0.6592573819641374, 'colsample_bytree': 0.6772046950185918, 'gamma': 3.133206736481381, 'reg_alpha': 0.545591764662148, 'reg_lambda': 0.00015066269333321563}. Best is trial 0 with value: 0.7969121449671592.
[I 2025-09-13 00:13:35,956] Trial 1 finished with value: 0.788079762744083 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 248, 'learning_rate': 0.24750593601210547, 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 4, 'subsample': 0.9799334637212604, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7969121449671592.
[I 2025-09-13 00:13:36,882] Trial 2 finished with value: 0.782041406030303 and parameters: {'classifier': 'RandomForest

F1 Score: 0.8013028915119647
Best hyperparameters: {'classifier': 'XGBoost', 'n_estimators': 190, 'max_depth': 8, 'learning_rate': 0.07726457339935157, 'subsample': 0.6591655840958631, 'colsample_bytree': 0.6622403999906079, 'gamma': 3.4176008553663326, 'reg_alpha': 0.16874213876228478, 'reg_lambda': 0.9023500820102178}


In [ ]:
study._get_best_trial()

## with smote 

In [51]:
from imblearn.combine import SMOTETomek

# Apply SMOTE-Tomek on training data only
smote_tomek = SMOTETomek(random_state=42, sampling_strategy="auto")
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train)

print("Before SMOTE-Tomek:", y_train.value_counts().to_dict())
print("After SMOTE-Tomek:", y_train_resampled.value_counts().to_dict())


Before SMOTE-Tomek: {(0,): 4162, (1,): 1425}
After SMOTE-Tomek: {(0,): 4028, (1,): 4028}


In [65]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier  # ✅ added

def objective(trial):
    classifier = trial.suggest_categorical("classifier", [
        "LogisticRegression", "RandomForest", "GradientBoosting", "KNN", "XGBoost"
    ])
    # add svc here 

    if classifier == "LogisticRegression":
        param = {
            'C': trial.suggest_float('C', 1e-4, 1e4, log=True), 
            'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga', 'newton-cg']), 
            'tol': trial.suggest_float('tol', 1e-6, 1e-1, log=True), 
            'class_weight': trial.suggest_categorical('lg_class_weight', [None, 'balanced'])
        }
        model = LogisticRegression(**param)

    elif classifier == "RandomForest":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'max_depth': trial.suggest_int("max_depth", 5, 30),
            'min_samples_split': trial.suggest_int("min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
            'random_state': 42,
            'n_jobs': -1
        }
        model = RandomForestClassifier(**param)

    elif classifier == "GradientBoosting":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int("max_depth", 2, 10),
            'min_samples_split': trial.suggest_int("min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
            'subsample': trial.suggest_float("subsample", 0.6, 1.0),
            'max_features': trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
            'random_state': 42
        }
        model = GradientBoostingClassifier(**param)

    # elif classifier == "SVC":
    #     param = {
    #         'C': trial.suggest_float("C", 0.1, 10, log=True),
    #         'kernel': trial.suggest_categorical("kernel", ["linear", "rbf"]),
    #         'gamma': trial.suggest_categorical("gamma", ["scale", "auto"]),
    #         'probability': True,
    #         'random_state': 42
    #     }
    #     model = SVC(**param)

    elif classifier == "KNN":
        param = {
            'n_neighbors': trial.suggest_int("n_neighbors", 3, 15),
            'weights': trial.suggest_categorical("weights", ["uniform", "distance"]),
            'metric': trial.suggest_categorical("metric", ["euclidean", "manhattan"])
        }
        model = KNeighborsClassifier(**param)

    elif classifier == "XGBoost":
        param = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'max_depth': trial.suggest_int("max_depth", 3, 10),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            'subsample': trial.suggest_float("subsample", 0.6, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
            'gamma': trial.suggest_float("gamma", 0, 5),
            'reg_alpha': trial.suggest_float("reg_alpha", 1e-4, 10, log=True),
            'reg_lambda': trial.suggest_float("reg_lambda", 1e-4, 10, log=True),
            'use_label_encoder': False,
            'eval_metric': 'logloss',
            'random_state': 42,
            'n_jobs': -1
        }
        model = XGBClassifier(**param)
        
    f1_scorer = make_scorer(f1_score, average='macro')
    return cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring=f1_scorer, n_jobs=-1).mean()

# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=60)

trial = study.best_trial

print("F1 Score: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))


[I 2025-09-13 00:14:21,086] A new study created in memory with name: no-name-7bc66491-cb46-400d-8898-c7038b10a43d
[I 2025-09-13 00:14:21,275] Trial 0 finished with value: 0.8088381721425526 and parameters: {'classifier': 'KNN', 'n_neighbors': 15, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 0 with value: 0.8088381721425526.
[I 2025-09-13 00:14:22,042] Trial 1 finished with value: 0.8906659005071402 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 126, 'learning_rate': 0.11826174454245682, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 3, 'subsample': 0.8229277307443134, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.8906659005071402.
[I 2025-09-13 00:14:22,272] Trial 2 finished with value: 0.8272944680382791 and parameters: {'classifier': 'KNN', 'n_neighbors': 8, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 1 with value: 0.8906659005071402.
[I 2025-09-13 00:14:22,437] Trial 3 finished with value: 0.8448445961069779 an

F1 Score: 0.8995553806341388
Best hyperparameters: {'classifier': 'GradientBoosting', 'n_estimators': 224, 'learning_rate': 0.08703333695306757, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 3, 'subsample': 0.7915313331316937, 'max_features': 'sqrt'}


In [66]:
study.best_trial

FrozenTrial(number=54, state=TrialState.COMPLETE, values=[0.8995553806341388], datetime_start=datetime.datetime(2025, 9, 13, 0, 16, 20, 223761), datetime_complete=datetime.datetime(2025, 9, 13, 0, 16, 23, 928873), params={'classifier': 'GradientBoosting', 'n_estimators': 224, 'learning_rate': 0.08703333695306757, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 3, 'subsample': 0.7915313331316937, 'max_features': 'sqrt'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('LogisticRegression', 'RandomForest', 'GradientBoosting', 'KNN', 'XGBoost')), 'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=5, log=False, low=1, step=1), 'subsample

In [68]:
from sklearn.metrics import classification_report

print('Best trial:')
print('  Recall: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

best_params = {k: v for k, v in study.best_params.items() if k != "classifier"}


best_model = GradientBoostingClassifier(**best_params)
best_model.fit(X_train_resampled, y_train_resampled)

# Evaluate on the test set
y_pred = best_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

Best trial:
  Recall: 0.8995553806341388
  Params: 
    classifier: GradientBoosting
    n_estimators: 224
    learning_rate: 0.08703333695306757
    max_depth: 10
    min_samples_split: 3
    min_samples_leaf: 3
    subsample: 0.7915313331316937
    max_features: sqrt


c:\Users\linto\Code\churn-x\.venv\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.81      0.87      0.84      1012
           1       0.58      0.48      0.52       385

    accuracy                           0.76      1397
   macro avg       0.70      0.67      0.68      1397
weighted avg       0.75      0.76      0.75      1397



In [47]:
    params = {
        "LogisticRegression": {
            'C': trial.suggest_float('lg_C', 1e-4, 1e4, log=True), 
            'solver': trial.suggest_categorical('lg_solver', ['lbfgs', 'liblinear', 'saga', 'newton-cg']), 
            'tol': trial.suggest_float('lg_tol', 1e-6, 1e-1, log=True), 
            'class_weight': trial.suggest_categorical('lg_class_weight', [None, 'balanced'])
        },

        "RandomForest": {
            'n_estimators': trial.suggest_int("rf_n_estimators", 100, 300),
            'max_depth': trial.suggest_int("rf_max_depth", 5, 30),
            'min_samples_split': trial.suggest_int("rf_min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("rf_min_samples_leaf", 1, 5),
            'random_state': 42,
            'n_jobs': -1
        },

        "GradientBoosting": {
            'n_estimators': trial.suggest_int("gb_n_estimators", 100, 300),
            'learning_rate': trial.suggest_float("gb_learning_rate", 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int("gb_max_depth", 2, 10),
            'min_samples_split': trial.suggest_int("gb_min_samples_split", 2, 10),
            'min_samples_leaf': trial.suggest_int("gb_min_samples_leaf", 1, 5),
            'subsample': trial.suggest_float("gb_subsample", 0.6, 1.0),
            'max_features': trial.suggest_categorical("gb_max_features", ["sqrt", "log2", None]),
            'random_state': 42
        },

        "SVC": {
            'C': trial.suggest_float("svc_C", 0.1, 10, log=True),
            'kernel': trial.suggest_categorical("svc_kernel", ["linear", "rbf"]),
            'gamma': trial.suggest_categorical("svc_gamma", ["scale", "auto"]),
            'probability': True,
            'random_state': 42
        },

        "KNN": {
            'n_neighbors': trial.suggest_int("knn_n_neighbors", 3, 15),
            'weights': trial.suggest_categorical("knn_weights", ["uniform", "distance"]),
            'metric': trial.suggest_categorical("knn_metric", ["euclidean", "manhattan"])
        }
    }

In [48]:
params["LogisticRegression"]

{'C': 0.05241530248680128,
 'solver': 'lbfgs',
 'tol': 0.0011552048342998223,
 'class_weight': None}